In [1]:
text = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""
lines = text.strip().split('\n')
lines

['seeds: 79 14 55 13',
 '',
 'seed-to-soil map:',
 '50 98 2',
 '52 50 48',
 '',
 'soil-to-fertilizer map:',
 '0 15 37',
 '37 52 2',
 '39 0 15',
 '',
 'fertilizer-to-water map:',
 '49 53 8',
 '0 11 42',
 '42 0 7',
 '57 7 4',
 '',
 'water-to-light map:',
 '88 18 7',
 '18 25 70',
 '',
 'light-to-temperature map:',
 '45 77 23',
 '81 45 19',
 '68 64 13',
 '',
 'temperature-to-humidity map:',
 '0 69 1',
 '1 0 69',
 '',
 'humidity-to-location map:',
 '60 56 37',
 '56 93 4']

In [2]:
from dataclasses import dataclass, field
from typing import List, Tuple

@dataclass
class Range:
    src_start: int
    src_end: int
    offset: int # can be negative

    def get(self, i: int):
        if i >= self.src_start and i <= self.src_end:
            return i + self.offset

@dataclass
class Mapping:
    name: str
    ranges: List[Range] = field(default_factory=list)

    def update(self, s: str):
        "take an individual line like 0 15 37 and update the internal map"
        dst, src, rng = [int(i) for i in s.split()]
        r = Range(src_start=src, src_end=src+rng, offset=dst-src)
        self.ranges.append(r)

    def get(self, i: int):
        for r in self.ranges:
            if new := r.get(i):
                return new
        return i

@dataclass
class Pipeline:
    maps: List[Mapping]

    @classmethod
    def from_lines(cls, lines: List[str]):
        "Parse a text blob to create a series of mappings"
        maps = []
        m = None
        for line in lines:
            line = line.strip()
            if line.endswith('map:'):
                name = line.split()[0]
                m = Mapping(name=name)
            elif not line:
                maps.append(m)
                m = None
            else:
                if m is None:
                    raise RuntimeError("got a parseable line but no Mapping obj to update: %s", line)
                m.update(line)
        maps.append(m)
        return cls(maps=maps)

    def get(self, i: int, debug=False):
        "Return the final value as i goes through each mapping in the pipeline"
        for mapping in self.maps:
            new = mapping.get(i)
            if debug:
                print(f"{i} -> {mapping.name} -> {new}")
            i = new
        return i

pipeline = Pipeline.from_lines(lines[2:])
for map in pipeline.maps:
    print(map.name, len(map.ranges))

seed-to-soil 2
soil-to-fertilizer 3
fertilizer-to-water 4
water-to-light 2
light-to-temperature 3
temperature-to-humidity 2
humidity-to-location 2


In [3]:
pipeline.get(79, debug=True)

79 -> seed-to-soil -> 81
81 -> soil-to-fertilizer -> 81
81 -> fertilizer-to-water -> 81
81 -> water-to-light -> 74
74 -> light-to-temperature -> 78
78 -> temperature-to-humidity -> 78
78 -> humidity-to-location -> 82


82

In [5]:
seeds = []
seed_numbers = lines[0].split(':')[1].split()
while seed_numbers:
    start = int(seed_numbers.pop(0))
    rng = int(seed_numbers.pop(0))
    for i in range(start, start+rng):
        seeds.append(i)

seeds

[79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67]

In [6]:
import math

answer = math.inf
for seed in seeds:
    result = pipeline.get(int(seed))
    print(seed, result)
    if result < answer:
        answer = result

answer

79 82
80 83
81 84
82 46
83 47
84 48
85 49
86 50
87 51
88 52
89 53
90 54
91 55
92 60
55 86
56 87
57 88
58 89
59 90
60 95
61 96
62 97
63 57
64 58
65 59
66 60
67 98


46

In [ ]:
import math
from dataclasses import dataclass, field
from typing import Dict, List

@dataclass
class Range:
    src_start: int
    src_end: int
    offset: int # can be negative

    def get(self, i: int):
        if i >= self.src_start and i <= self.src_end:
            return i + self.offset

@dataclass
class Mapping:
    name: str
    ranges: List[Range] = field(default_factory=list)

    def update(self, s: str):
        "take an individual line like 0 15 37 and update the internal map"
        dst, src, rng = [int(i) for i in s.split()]
        r = Range(src_start=src, src_end=src+rng, offset=dst-src)
        self.ranges.append(r)

    def get(self, i: int):
        for r in self.ranges:
            if new := r.get(i):
                return new
        return i

@dataclass
class Pipeline:
    maps: List[Mapping]

    @classmethod
    def from_lines(cls, lines: List[str]):
        "Parse a text blob to create a series of mappings"
        maps = []
        m = None
        for line in lines:
            line = line.strip()
            if line.endswith('map:'):
                name = line.split()[0]
                m = Mapping(name=name)
            elif not line:
                maps.append(m)
                m = None
            else:
                if m is None:
                    raise RuntimeError("got a parseable line but no Mapping obj to update: %s", line)
                m.update(line)
        maps.append(m)
        return cls(maps=maps)

    def get(self, i: int, debug=False):
        "Return the final value as i goes through each mapping in the pipeline"
        for mapping in self.maps:
            new = mapping.get(i)
            if debug:
                print(f"{i} -> {mapping.name} -> {new}")
            i = new
        return i


lines = open('../data/day05.txt').readlines()
pipeline = Pipeline.from_lines(lines[2:])

seeds = []
seed_numbers = lines[0].split(':')[1].split()
while seed_numbers:
    start = int(seed_numbers.pop(0))
    rng = int(seed_numbers.pop(0))
    for i in range(start, start+rng):
        seeds.append(i)

answer = math.inf
for seed in seeds:
    result = pipeline.get(int(seed))
    if result < answer:
        answer = result
answer        